In [1]:
import requests
URL = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
DOCUMENT_NAME = "sample_document.txt"

res = requests.get(URL)
with open(DOCUMENT_NAME, 'w') as f:
    f.write(res.text)

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader(DOCUMENT_NAME)
documents = loader.load()

In [3]:
FAISS_INDEX_PATH = "faiss_index"

TEXT_SPLITTER_CHUNK_SIZE = 1000
TEXT_SPLITTER_CHUNK_OVERLAP = 0

HUGGINGFACE_MODELNAME = "google/flan-t5-base"
HUGGINGFACE_MODEL_KWARGS = {"temperature":0, "max_length":512}

LANGCHAIN_QNA_CHAIN_TYPE = "stuff"

In [4]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=TEXT_SPLITTER_CHUNK_SIZE,
                                      chunk_overlap=TEXT_SPLITTER_CHUNK_OVERLAP)
docs = text_splitter.split_documents(documents)

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

d:\InstalledSoftware\anaconda3\envs\LangchainApp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [7]:
query = "What did the president say about the Supreeme Court"
docs = db.similarity_search(query)

In [8]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [9]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And
while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an
Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer,
thank you for your service.

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the
United States Supreme Court.

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our
nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [10]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [11]:
import dotenv
dotenv.load_dotenv()

True

In [12]:
llm=HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature":0, "max_length":512})

d:\InstalledSoftware\anaconda3\envs\LangchainApp\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [13]:
chain = load_qa_chain(llm, chain_type="stuff")

In [14]:
query = "What did the president say about the Supreme Court"
docs = db.similarity_search(query)
chain.run(input_documents=docs, question=query)

d:\InstalledSoftware\anaconda3\envs\LangchainApp\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Justice Breyer'

In [15]:
query = "What did the president say about economy?"
docs = db.similarity_search(query)
data = chain.invoke({"input_documents":docs, "question":query})

In [16]:
db.save_local(FAISS_INDEX_PATH)

In [17]:
new_db = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

In [18]:
query = "What did the president say about the Supreme Court"
docs = new_db.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Justice Breyer'